In [3]:
import gym # Imports gym

env = gym.make("Taxi-v3").env #Creates taxi v3 enviroment 

env.render() # Renders current enviroment

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))


Action Space Discrete(6)
State Space Discrete(500)


Actions:

0 = south
1 = north
2 = east
3 = west
4 = pickup
5 = dropoff

In [5]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
env.P[328] # Reward Values assigned to current state

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

Format: action: [(probability, nextstate, reward, done)]
Probability Always 1.0 in this enviroment
nextstate is the state taken with the action performed, eg 0 being moving south.

In [7]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
#print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Penalties incurred: 615


In [8]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
#print_frames(frames)

In [9]:
import numpy as np
# Setting up the Q-Table, with values for each possible position and action. (500 x 6 here)

q_table = np.zeros([env.observation_space.n, env.action_space.n])
print(len(q_table))

import random
from IPython.display import clear_output

500


In [10]:
# Hyperparameters
alpha = 0.1 # Learning Rate, should decrease with larger knowledge bases
gamma = 0.6 # Should decrease as the deadline approaches as long-term rewards become less important
epsilon = 0.1 # Exploration vs exploitation, should decrease as the trials increase

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



In [11]:
q_table[328]

array([ -2.40410547,  -2.27325184,  -2.40364156,  -2.35219864,
       -10.32278892, -10.3093861 ])

In [12]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.86
Average penalties per episode: 0.0


To tune the hyperparameters, maybe make a grid search that selects parameters that result in the best reward/time steps ratio.
More fancy way is genetic algorithms.

In [25]:
done = False
frames = []
epochs = 0
state = env.reset()

history_dict = dict()


print(q_table[state])
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    if epochs > 100:
        break
    
print_frames(frames)

#print(q_table)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 13
State: 410
Action: 5
Reward: 20


Tutorial Link:
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

Strengths:
- Easy to implement
- Good for small sample spaces with few actions
Weaknesses:
- Requires a large amount of memory for larger sample spaces
- Slow

Algorithms to try next:
- Value Iteration
- Policy Iteration
- Proximal Policy Algorithm
- Deep Learning